https://www.tensorflow.org/guide/eager#object_based_saving

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

from TPM_from_VCS.data import toy_data_generator

In [3]:
class Model(tf.keras.Model):
  def __init__(self):
    super(Model, self).__init__()
    self.W = tf.Variable(tf.random_normal((2, 5), dtype=tf.float64), name='weight')
    self.B = tf.Variable(tf.random_normal((2, 1), dtype=tf.float64), name='bias')

    
  def call(self, inputs):
    return tf.add(tf.matmul(self.W, tf.reshape(inputs, (5, -1))), self.B)

# The loss function to be optimized
def loss(model, inputs, targets):
  error = model(inputs) - targets
  return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])

# Define:
# 1. A model.
# 2. Derivatives of a loss function with respect to model parameters.
# 3. A strategy for updating the variables based on the derivatives.
model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

a = toy_data_generator.create_dataset(1)
training_inputs = a[0][0]
training_outputs = a[0][1]

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# Training loop
for j in range(10):
    for i in range(300):
      grads = grad(model, training_inputs, training_outputs)
      optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                                global_step=tf.train.get_or_create_global_step())
      if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))
        
    a = toy_data_generator.create_dataset(1)
    training_inputs = a[0][0]
    training_outputs = a[0][1]

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

Number of nodes currently : 10
Completed VC processing
Initial loss: 2557128.432
Loss at step 000: 1022760262668704.000
Loss at step 020: 11225358580246348943124417642887122486792347665672283866342921301286046680347582861779772922772140758340188588160469612503646990997228885459317252190862829807000293478129647326853473501184.000
Loss at step 040: inf
Loss at step 060: inf
Loss at step 080: nan
Loss at step 100: nan
Loss at step 120: nan
Loss at step 140: nan
Loss at step 160: nan
Loss at step 180: nan
Loss at step 200: nan
Loss at step 220: nan
Loss at step 240: nan
Loss at step 260: nan
Loss at step 280: nan
Number of nodes currently : 36
Completed VC processing
Loss at step 000: nan
Loss at step 020: nan
Loss at step 040: nan
Loss at step 060: nan
Loss at step 080: nan
Loss at step 100: nan
Loss at step 120: nan
Loss at step 140: nan
Loss at step 160: nan
Loss at step 180: nan
Loss at step 200: nan
Loss at step 220: nan
Loss at step 240: nan
Loss at step 260: nan
Loss at step 280: na

https://www.tensorflow.org/guide/summaries_and_tensorboard   : use tensorboard

https://stackoverflow.com/questions/40050397/deep-learning-nan-loss-reasons